In [39]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [40]:
from llama_index.llms.llama_cpp import LlamaCPP

model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    model_url=model_url,
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from C:\Users\Administrator\AppData\Local\llama_index\models\llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:        

In [27]:
from llama_index.readers.file import PyMuPDFReader

In [28]:


loader = PyMuPDFReader()
documents = loader.load(file_path="pdfs/RAG.pdf")


In [ ]:
from llama_index.core.node_parser import SentenceSplitter
text_parser = SentenceSplitter(
    chunk_size=500,
    # separator=" ",
)
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))
    
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)
    
    
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding
    
vector_store.add(nodes)   


In [32]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)
query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

query_str = "What are briefly the basics of RAG?"

response = query_engine.query(query_str)

print(response)

In [41]:
import weaviate
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader



client = weaviate.connect_to_local(
    host="localhost",  
    port=8080,
    grpc_port=50051,
)
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name="LlamaIndex"
)


In [42]:
documents = SimpleDirectoryReader("pdfs").load_data()

In [43]:
index = VectorStoreIndex.from_documents(
    documents,
    vector_store=vector_store,
    embed_model=embed_model
)

query_engine = index.as_query_engine(llm=llm, embed_model=embed_model)


In [44]:
response = query_engine.query("What are the difficulties when applying RAG?")
print(response)

llama_perf_context_print:        load time =  282413.64 ms
llama_perf_context_print: prompt eval time =  282411.71 ms /  2004 tokens (  140.92 ms per token,     7.10 tokens per second)
llama_perf_context_print:        eval time =   93043.68 ms /   107 runs   (  869.57 ms per token,     1.15 tokens per second)
llama_perf_context_print:       total time =  375912.55 ms /  2111 tokens


 There are several potential downsides to applying RAG, including the possibility of
generating abuse, faked, or misleading content in the news or on social media, impersonating
others, or automating the production of spam/phishing content. Additionally, advanced language
models may lead to the automation of various jobs in the coming decades. To mitigate these risks,
AI systems could be employed to fight against misleading content and automated spam/phishing.
